<a href="https://colab.research.google.com/github/Clva4454/CSPB/blob/main/NLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dataset

In [22]:
!pip install yake
!pip install sentence-transformers

In [18]:
import re
import pandas as pd
import numpy as np
import nltk
import os

from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [32]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Download latest version
# path = kagglehub.dataset_download("sunnysai12345/news-summary")

path = load_dataset("abisee/cnn_dailymail", "1.0.0")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
dataset = load_dataset("abisee/cnn_dailymail", "1.0.0")
df = dataset["train"].to_pandas()

# Remove ID
news_data = df.drop(columns=["id"])

# Keep first 50 for testing
subset = news_data.head(50).copy()

subset.head()

article  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost

In [43]:
# Clean and inspect data loaded.

# Check for duplicates
duplicate_rows = df[df.duplicated()]
# print("Duplicate rows found:", len(duplicate_rows))

# Remove noise: drop only 'id'
pd.set_option('display.max_colwidth', None)
news_data = df.drop(['id'], axis=1)

print(news_data.head())

In [23]:
display(news_data.head(50))

article  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# TF-IDF

In [24]:
class TfidfSummaryModel:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(stop_words=list(stopwords.words('english')))

    # Helpers:.
    def fix_punctuation(self, text):
        text = str(text)
        # Add space after period.
        text = re.sub(r'\.([A-Z])', r'. \1', text) # Corrected regex to only add space before uppercase letters
        return text

    def split_sentences(self, article):
        article = str(article)
        article = self.fix_punctuation(article)
        sentences = sent_tokenize(article)

        return sentences

    def score_sentences(self, sentences):
        tfidf_matrix = self.vectorizer.fit_transform(sentences)
        scores = tfidf_matrix.mean(axis=1).A1

        return scores

    def summary(self, article, total_sentences = 3):
            sentences = self.split_sentences(article)

            if len(sentences) <= total_sentences:
                return " ".join(sentences)

            # Select top sentences.
            scores = self.score_sentences(sentences)
            top_idx = scores.argsort()[-total_sentences:][::-1]
            top_sentences = [sentences[i] for i in sorted(top_idx)]

            summary = " ".join(top_sentences)

            return summary

In [25]:
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)

tfidf_model = TfidfSummaryModel()

for i in range(3):
    article = news_data['article'].iloc[i]
    sentences = tfidf_model.split_sentences(article)
    for j, s in enumerate(sentences):
        print(f"[{j}] {s}")

[0] LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
[1] Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
[2] "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.
[3] "I don't think I'll be particularly extravagant.
[4] "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."
[5] At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box offi

In [26]:
tfidf_model = TfidfSummaryModel()

for i in range(3):
    article = news_data['article'].iloc[i]
    tfidf_model.summary(article)

In [27]:
data_including_tfidf = news_data.head(75).copy()

tfidf_model = TfidfSummaryModel()
data_including_tfidf["tfidf_summary"] = data_including_tfidf["article"].apply(
    lambda a: tfidf_model.summary(a))

display(data_including_tfidf[["article", "highlights", "tfidf_summary"]])

article  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# BART Model

In [46]:
bart = pipeline("summarization", model="facebook/bart-large-cnn", device="cuda")

subset["bart_summary"] = subset["article"].apply(
    lambda a: bart(a[:1024], max_length=130, min_length=30, truncation=True)[0]["summary_text"])

Device set to use cuda
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# Hybrid Model

In [38]:
class HybridSummaryModel:
    def __init__(self, tfidf_model, bert_model=None):
        self.tfidf_model = tfidf_model
        self.bert_model = bert_model or SentenceTransformer("distilbert-base-nli-mean-tokens")

    def semantic_scores(self, article, sentences):
        article_emb = self.bert_model.encode([article])
        sent_embs = self.bert_model.encode(sentences)

        sims = cosine_similarity(article_emb, sent_embs)[0]
        return sims

    def hybrid_scores(self, tfidf_scores, semantic_scores, tfidf_weight = 0.5, semantic_weight = 0.5):
        tfidf_normalized = tfidf_scores / (tfidf_scores.max() + 1e-8)
        semantic_normalized = semantic_scores / (semantic_scores.max() + 1e-8)

        return (tfidf_weight * tfidf_normalized) + (semantic_weight * semantic_normalized)

    def summary(self, article, total_sentences = 3, tfidf_weight = 0.5, semantic_weight = 0.5):
        sentences = self.tfidf_model.split_sentences(article)

        if len(sentences) <= total_sentences:
            return " ".join(sentences) # Too short to summarize.

        tfidf_scores = self.tfidf_model.score_sentences(sentences)

        semantic_scores = self.semantic_scores(article, sentences)

        combined_hybrid_score = self.hybrid_scores(tfidf_scores, semantic_scores, tfidf_weight, semantic_weight)

        top_k_sentences = combined_hybrid_score.argsort()[-total_sentences:][::-1]
        top_sentences = [sentences[i] for i in sorted(top_k_sentences)]

        return " ".join(top_sentences)


In [47]:
model = SentenceTransformer("distilbert-base-nli-mean-tokens")

def semantic_sentence_scores(article, sentences):
    article_emb = model.encode([article])
    sent_embs = model.encode(sentences)

    sims = cosine_similarity(article_emb, sent_embs)[0]
    return sims

In [48]:
subset["hybrid_summary"] = subset["article"].apply( lambda a: hybrid.summary(a))

In [49]:
class HybridAbstractiveModel:
    def __init__(self, hybrid_extractive_model, bart_model):
        self.hybrid_extractive = hybrid_extractive_model
        self.bart = bart_model

    def final_summary(self, article):
        extractive = self.hybrid_extractive.summary(article)

        rewritten_extractive_with_BART = self.bart(extractive, max_length=120, min_length=30, truncation=True)[0]["summary_text"]

        return rewritten_extractive_with_BART

In [50]:
hybrid_bart = HybridAbstractiveModel(hybrid, bart)

subset["hybrid_bart_summary"] = subset["article"].apply(lambda a: hybrid_bart.final_summary(a))

Your max_length is set to 120, but your input_length is only 114. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


In [51]:
display(subset[[
    "highlights",
    "tfidf_summary",
    "bart_summary",
    "hybrid_summary",
    "hybrid_bart_summary"
]])

KeyError: "['tfidf_summary'] not in index"

In [61]:
display(subset[["highlights", "tfidf_summary", "bart_summary", "hybrid_summary", "hybrid_bart_summary"]])

,highlights,tfidf_summary,bart_summary,hybrid_summary,hybrid_bart_summary
0,Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five Potter films have been held in trust fund .,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in ""Harry Potter and the Order of the Phoenix"" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film ""Hostel: Part II,"" currently six places below his number one movie on the UK box office chart.","Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe says he has no plans to fritter his cash away on fast cars, drink.","LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in ""Harry Potter and the Order of the Phoenix"" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film ""Hostel: Part II,"" currently six places below his number one movie on the UK box office chart.",Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe says he has no plans to fritter his cash away.
1,"Mentally ill inmates in Miami are housed on the ""forgotten floor"" Judge Steven Leifman says most are there as a result of ""avoidable felonies"" While CNN tours facility, patient shouts: ""I am the son of the president"" Leifman says the system is unjust and he's fighting for change .","Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman. Leifman says in 1955 there were more than half a million people in state mental hospitals, and today that number has been reduced 90 percent, and 40,000 to 50,000 people are in mental hospitals. Starting in 2008, many inmates who would otherwise have been brought to the ""forgotten floor"" will instead be sent to a new mental health facility -- the first step on a journey toward long-term treatment, not just punishment.","Miami-Dade pretrial detention facility houses mentally ill inmates. Most often, they face drug charges or charges of assaulting an officer. Judge: Arrests often result from confrontations with police.","Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the ""forgotten floor,"" where many mentally ill inmates are housed in Miami before trial. Leifman tells me that these prisoner-patients will often circulate through the system, occasionally stabilizing in a mental hospital, only to return to jail to face their charges. Starting in 2008, many inmates who would otherwise have been brought to the ""forgotten floor"" will instead be sent to a new mental health facility -- the first step on a journey toward long-term treatment, not just punishment.","Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. Starting in 2008, many inmates who would otherwise have been brought to the ""forgotten floor"" will instead be sent to a new mental health facility."
2,"NEW: ""I thou

# ROUGE Evaluation

In [62]:
pip install rouge-score

In [63]:
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

results = []

for i in range(len(subset)):
    reference = subset["highlights"].iloc[i]
    summaries = {
        "TF-IDF": subset["tfidf_summary"].iloc[i],
        "BART": subset["bart_summary"].iloc[i],
        "Hybrid": subset["hybrid_summary"].iloc[i],
        "Hybrid+BART": subset["hybrid_bart_summary"].iloc[i], }


In [58]:
from rouge_score import rouge_scorer
import pandas as pd

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

results = []

In [64]:
for i in range(len(subset)):

    reference = subset["highlights"].iloc[i]

    models = {
        "Hybrid": subset["hybrid_summary"].iloc[i],
        "BART": subset["bart_summary"].iloc[i],
        "HybridBART": subset["hybrid_bart_summary"].iloc[i],
    }

    for model_name, summary in models.items():
        score = scorer.score(reference, summary)

        results.append({
            "article": i + 1,
            "model": model_name,
            "ROUGE-1": score["rouge1"].fmeasure,
            "ROUGE-2": score["rouge2"].fmeasure,
            "ROUGE-L": score["rougeL"].fmeasure,
        })

rouge_df = pd.DataFrame(results)
display(rouge_df)

,article,model,ROUGE-1,ROUGE-2,ROUGE-L
0,1,Hybrid,0.341772,0.243590,0.316456
1,1,BART,0.560000,0.383562,0.480000
2,1,HybridBART,0.591549,0.405797,0.507042
3,2,Hybrid,0.280000,0.135135,0.173333
4,2,BART,0.256410,0.052632,0.128205
...,...,...,...,...,...
145,49,BART,0.505495,0.247191,0.219780
146,49,HybridBART,0.358974,0.184211,0.230769
147,50,Hybrid,0.242424,0.046154,0.166667
148,50,BART,0.205882,0.030303,0.147059
